Linear Models: Model simplification {#chap:ModelSimp}
===================================

Aims of this chapter[^1]:

Simplifying complex models by removing non-explanatory terms

In biology, we often use statistics to compare competing hypotheses in
order to work out the simplest explanation for some data. This often
involves collecting several explanatory variables that describe
different hypotheses and then fitting them together in a single model,
and often including interactions between those variables.

In all likelihood, not all of these model <span>*terms*</span> will be
important. If we remove unimportant terms, then the explanatory power of
the model will get worse, but might not get significantly worse.

> *“It can scarcely be denied that the supreme goal of all theory is to
> make the irreducible basic elements as simple and as few as possible
> without having to surrender the adequate representation of a single
> datum of experience.”*
>
> *Albert Einstein*

Or to paraphrase:

> *“Everything should be made as simple as possible, but no simpler.”*

The approach we will look at is to start with a <span>*maximal
model*</span> — the model that contains everything that might be
important — and simplify it towards the <span>*null model*</span> — the
model that says that none of your variables are important. Hopefully,
there is a point somewhere in between where you can’t remove any further
terms without making the model significantly worse: this is called the
<span>*minimum adequate model*</span>.

\[every node/.style=<span>draw=black!70, very thick, rectangle, rounded
corners</span>\] (a) at (0,0) <span>Maximal model</span>; (b) at (5,0)
<span>Minimum adequate model</span>; (c) at (10,0) <span>Null
model</span>; (a.east) – (b.west); (b.east) – (c.west);

A maximal model
---------------

We’ll be using the mammal dataset for this practical, so once again:

\[$\quad\star$\]

Make sure you have changed the working directory to your stats module
<span>Code</span> folder.

Create a new blank script called ‘MyModelSimp.R’.

Load the mammals data into a data frame called <span>mammals</span>.

In Chapters 5 & \[ch:MulExplInter\], we looked at how the categorical
variables <span>GroundDwelling</span> and <span>TrophicLevel</span>
predicted genome size in mammals. In this chapter, we will add in two
more continuous variables: litter size and body mass. The first thing we
will do is to log both variables and reduce the dataset to the rows for
which all of these data are available:

In [ ]:
#get logs of continuous variables
> mammals$logLS <- log(mammals$LitterSize)
> mammals$logCvalue <- log(mammals$meanCvalue)
> mammals$logBM <- log(mammals$AdultBodyMass_g)

# reduce dataset to five key variables
> mammals <- subset(mammals, select = c(logCvalue, logLS, logBM, 
TrophicLevel, GroundDwelling))

# remove the row with missing data
> mammals <- na.omit(mammals)

\[$\quad\star$\]

Copy the code above into your script and run it

Check that the data you end up with has this structure:

In [ ]:
'data.frame':  240 obs. of  5 variables:
$logCvalue     : num  0.94 1.322 1.381 1.545 0.888 ...
$logLS         : num  1.1 1.12 0 0 1.52 ...
$logBM         : num  10.83 4.87 11.46 10.86 3.23 ...
$TrophicLevel  : Factor w/ 3 levels "Carnivore","Herbivore",..: 1 2 2 2 3 3 3 2 2 3 ...
$GroundDwelling: Factor w/ 2 levels "No","Yes": 2 2 2 2 2 1 2 1 1 1 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:139] 2 4 7 9 10 11 14 15 20 21 ...
  .. ..- attr(*, "names")= chr [1:139] "2" "4" "7" "9" ...

Now we’ll fit a model including all of these variables and all of the
interactions:

In [ ]:
> model <- lm(formula = logCvalue ~ logLS * logBM * TrophicLevel * 
GroundDwelling, data = mammals)

\[$\quad\star$\]

Run this model in your script.

Look at the output of <span>anova(model)</span> and <span>
summary(model)</span>.

Scared? Don’t be! There are a number of points to this exercise:

1.  These tables show exactly the kind of output you’ve seen before.
    Sure, there are lots of rows but each row is just asking whether a
    model term (<span>anova</span>) or a model coefficient (<span>
    summary</span>) is significant.

2.  Some of the rows are significant, others aren’t: some of the model
    terms are not explanatory.

3.  The two tables show slightly different things - lots of stars for
    the <span>anova</span> table and only a few for the
    <span>summary</span> table.

4.  That last line in the <span>anova</span> table: <span>
    logLS:logBM:TrophicLevel:GroundDwelling</span>. This is an
    interaction of four variables capturing how the slope for litter
    size changes for different body masses for species in different
    trophic groups and which are arboreal or ground dwelling. Does this
    seem easy to understand?

The real lesson here is that it is easy to fit complicated models in R.
<span>*Understanding and explaining them is a different matter*</span>.
The temptation is always to start with the most complex possible model
but this is rarely a good idea.

A better maximal model
----------------------

Instead of all possible interactions, we’ll consider two-way
interactions: how do pairs of variables affect each other? There is a
shortcut for this: <span>y \~ (a + b + c)\^2</span> gets all two way
combinations of the variables in the brackets, so is a quicker way of
getting this model:

<span>y \~ a + b + c + a:b + a:c + b:c</span>.

So let’s use this to fit a simpler maximal model:

In [ ]:
> model <- lm(logCvalue ~ (logLS + logBM + TrophicLevel + GroundDwelling)^2, data = mammals)

The <span>anova</span> table for this model looks like this:

In [ ]:
> anova(model)

 Analysis of Variance Table
 
 Response: logCvalue
                              Df Sum Sq Mean Sq F value  Pr(>F)    
 logLS                         1   0.99   0.989   25.72 8.2e-07 ***
 logBM                         1   3.03   3.032   78.83 < 2e-16 ***
 TrophicLevel                  2   0.48   0.239    6.21  0.0024 ** 
 GroundDwelling                1   0.11   0.110    2.87  0.0915 .  
 logLS:logBM                   1   0.27   0.275    7.15  0.0081 ** 
 logLS:TrophicLevel            2   0.19   0.095    2.48  0.0862 .  
 logLS:GroundDwelling          1   0.14   0.136    3.55  0.0609 .  
 logBM:TrophicLevel            2   0.09   0.044    1.14  0.3230    
 logBM:GroundDwelling          1   0.88   0.883   22.96 3.0e-06 ***
 TrophicLevel:GroundDwelling   2   0.04   0.022    0.58  0.5607    
 Residuals                   225   8.65   0.038                    
 ---
 Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

The first lines are the <span>*main effects*</span>, which are all
significant or near significant. Then there are the six interactions.
One of these is very significant: <span>logBM:GroundDwelling</span>,
which suggests that the slope of log C value with body mass differs
between ground dwelling and non-ground dwelling species. The other
interactions are non-significant although some are close.

\[$\quad\star$\]

Run this model in your script.

Look at the output of <span>anova(model)</span> and <span>
summary(model)</span>.

Check the model diagnostic plots.

Model simplification
--------------------

Model simplification is not a simple process. Each time you remove a
term from a model, the model will change: the model will get worse,
since some of the sums of squares are no longer explained, but the
remaining variables may take over.

The first question is: <span>*what terms can you remove from a
model*</span>? Obviously, you only want to remove non-significant terms,
but there is another rule – you cannot remove a main effect or an
interaction while those main effects or interactions are present in a
more complex interaction. For example, in the model <span>y \~ a + b + c
+ a:b + a:c + b:c</span>, you cannot drop <span>c</span> without
dropping both <span>a:c</span> and <span>b:c</span>.

The R function <span>drop.scope</span> tells you what you can drop from
a model. Some examples:

In [ ]:
> drop.scope(model)
 [1] "logLS:logBM"                 "logLS:TrophicLevel"         
 [3] "logLS:GroundDwelling"        "logBM:TrophicLevel"         
 [5] "logBM:GroundDwelling"        "TrophicLevel:GroundDwelling"

> drop.scope(y ~ a + b + c + a:b)
 [1] "c"   "a:b"
> drop.scope(y ~ a + b + c + a:b + b:c + a:b:c)
 [1] "a:b:c"

Model simplification is an iterative process. The flow diagram below
shows how it works: at each stage you try and find an acceptable
simplification. If successful, then you start again with the new simpler
model and try and find a way to simplify this, until eventually, you
can’t find anything more to remove.

\[bx/.style=<span>draw=black!70, very thick, rectangle, align=center,
rounded corners, fill=black!10, minimum width=4 cm</span>, every
path/.style=<span>draw=black!70, very thick, -stealth, shorten &gt;=
1mm</span>, node distance=0.8cm\] (a) at (0,0) <span>**Maximal
model**</span>; (b) <span>Current model</span>; (b2) <span>Make list of
valid\
terms to drop</span>; (c) <span>Remove the least-\
significant valid term</span>; (d) <span>New model</span>; (e)
<span>Compare current\
and new models</span>; (f) <span>Is the new\
model significantly\
worse?</span>; (g) <span>Remove current\
term from list\
of valid terms</span>; (h) <span>Are there\
other valid terms\
to drop?</span>; (i) <span>**Minimum adequate model**</span>; (j)
<span>New model becomes\
current model</span>;

(a.south) – (b.north); (b.south) – (b2.north); (b2.south) – (c.north);
(c.south) – (d.north); (d.south) – (e.north); (e.south) – (f.north);
(f.west) -| node\[below\]<span>No</span> (j.south) ; (j.north) |-
(b.west) ; (f.east) – node\[below\]<span>Yes</span> (g.west); (g.north)
– (h.south); (h.north) |- node\[right,
pos=0.5\]<span>Yes</span>(c.east); (h.east) –
node\[below\]<span>No</span> ++(1,0) |- (i.east);

As always, we can use an $F$ test to compare two models and see if they
have significantly different explanatory power. Here, significance is a
bad thing — it means that we’ve removed a term that makes the model
significantly worse.

The last thing we need to do is work out how to remove a term from a
model. We could type out the model again, but there is a shortcut using
the function <span>update</span>:

In [ ]:
# a simple model
> f <- y ~ a + b + c + b:c

# remove b:c from the current model
> update(f, . ~ . - b:c)
 y ~ a + b + c
 
# model g as a response using the same explanatory variables.
> update(f, g ~ .)
 g ~ a + b + c + b:c

Yes, the syntax is a little odd. The function uses a model or a formula
and then allows you to alter the current formula. The dots in the code
<span>. \~ . </span> mean ‘use whatever is currently in the response or
explanatory variables’. It gives a simple way of changing a model.

Putting this together, let’s try a simplification. From the previous
<span>anova</span> and <span>drop.scope</span> output, we know that the
interaction <span>TrophicLevel:GroundDwelling</span> is not significant
and a valid term.

In [ ]:
# remove TrophicLevel:GroundDwelling
> model2 <- update(model, . ~ . - TrophicLevel:GroundDwelling)

# use anova to compare the two models
> anova(model, model2)

 Analysis of Variance Table
 
 Model 1: logCvalue ~ (logLS + logBM + TrophicLevel + GroundDwelling)^2
 Model 2: logCvalue ~ logLS + logBM + TrophicLevel + GroundDwelling + 
            logLS:logBM + logLS:TrophicLevel + logLS:GroundDwelling + 
            logBM:TrophicLevel + logBM:GroundDwelling
   Res.Df  RSS Df Sum of Sq    F Pr(>F)
 1    225 8.65                         
 2    227 8.70 -2   -0.0446 0.58   0.56

This tells us that <span>model2</span> is not significantly worse than
<span> model</span>. We can now look at this model and see what else can
be removed:

In [ ]:
> anova(model2)
 
 Analysis of Variance Table
 
 Response: logCvalue
                       Df Sum Sq Mean Sq F value  Pr(>F)    
 logLS                  1   0.99   0.989   25.82 7.8e-07 ***
 logBM                  1   3.03   3.032   79.12 < 2e-16 ***
 TrophicLevel           2   0.48   0.239    6.24  0.0023 ** 
 GroundDwelling         1   0.11   0.110    2.88  0.0909 .  
 logLS:logBM            1   0.27   0.275    7.17  0.0079 ** 
 logLS:TrophicLevel     2   0.19   0.095    2.49  0.0854 .  
 logLS:GroundDwelling   1   0.14   0.136    3.56  0.0604 .  
 logBM:TrophicLevel     2   0.09   0.044    1.14  0.3216    
 logBM:GroundDwelling   1   0.88   0.883   23.05 2.9e-06 ***
 Residuals            227   8.70   0.038                    
 ---
 Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 
 
> drop.scope(model2)
 
 [1] "logLS:logBM"          "logLS:TrophicLevel"   "logLS:GroundDwelling"
 [4] "logBM:TrophicLevel"   "logBM:GroundDwelling"

\[$\quad\star$\]

Run this first simplification in your script.

Look at the output above and decide what is the next possible term to
delete

Using the code above as a model, create <span>model3</span> as the next
simplification! (remember to use <span>model2</span> in your update call
and not <span>model</span>)

Now for a difficult exercise:

\[$\quad\star$\]

Using the code above to guide you, try and find a minimal adequate model
that you are happy with. In each step, the output of <span>anova(model,
modelN)</span> should be non-significant (where $N$ is the current
step).

It can be important to consider both <span>anova</span> and <span>
summary</span> tables. It can be worth trying to remove things that look
significant in one table but not the other — some terms can explain
significant variation on the <span>anova</span> table but the
coefficients are not significant.

Remember to remove <span>*terms*</span>: with categorical variables,
several coefficients in the <span>summary</span> table may come from one
term in the model and have to be removed together.

When you have got your final model, save the model as an R data file:\
<span>save(modelN, file=’myFinalModel.Rda’)</span>.

[^1]: Here you work with the script file <span>ModelSimp.R</span>